In [29]:
import mlflow
import mlflow.sklearn
import pandas as pd
from numpy import savetxt
from sklearn.datasets import load_iris
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, get_scorer_names, accuracy_score,confusion_matrix, roc_curve
from mlflow import log_metric, log_param, log_artifacts

from pprint import pprint
import mlflow

import seaborn as sn
import matplotlib.pyplot as plt


In [5]:
fradues_df = pd.read_csv('datos_regiones_especiales_201906-202205.csv').drop(['grupo','NombreRegion'], axis = 1)#.drop(['LOCALIDAD'], axis = 1)

In [4]:
fradues_df.head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,...,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
0,1002467,122.72,111.53,113.11,97.14,102.40,95.17,121.67,125.47,133.44,...,124.71,123.69,1784.0,CHIMALTENANGO,CHIMALTENANGO,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
1,1017184,0.00,3.15,10.46,114.57,1.01,1.90,1.05,0.95,0.00,...,0.00,0.00,1779.0,CHIMALTENANGO,SAN MARTIN JILOTEPEQUE,Residenciales,Presunto Fraude,0,CentroOccidente II,1
2,2405770,111.18,147.18,161.90,236.47,344.72,323.79,90.20,81.74,122.65,...,270.71,245.36,2530.0,SOLOLA,SANTA LUCIA UTATLAN,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
3,2442128,58.87,62.93,94.19,294.35,227.11,207.21,205.57,212.92,238.58,...,255.50,208.99,2033.0,QUICHE,SANTA CRUZ DEL QUICHE,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
4,3161851,25.51,13.64,2.94,11.41,11.54,13.31,23.07,27.57,39.25,...,42.58,41.21,921.0,SANTA ROSA,CUILAPA,Residenciales,Tapa rota/falta,0,SurOriente,1


### Scale data

In [6]:
scaled_features = fradues_df.copy()

In [20]:
scaled_cols=list(scaled_features.columns[1:14])
scaled_cols.append('anomalias_anuales')
scaled_cols

['202106',
 '202107',
 '202108',
 '202109',
 '202110',
 '202111',
 '202112',
 '202201',
 '202202',
 '202203',
 '202204',
 '202205',
 'AlturaPuntoConsumo',
 'anomalias_anuales']

In [23]:
#
col_names = list(scaled_features.columns[1:14])
col_names.append('anomalias_anuales')
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features[col_names] = features
scaled_features.head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,DescripcionAnomalia,anomalias_anuales,fraude
0,1002467,-0.048889,-0.057541,-0.059095,-0.078472,-0.070510,-0.084197,-0.065306,-0.060530,-0.051858,-0.078475,-0.075367,-0.075686,1.121773,CHIMALTENANGO,CHIMALTENANGO,Residenciales,NO REGSITRADO,-0.372017,1
1,1017184,-0.159881,-0.154602,-0.154399,-0.062401,-0.162218,-0.176305,-0.177218,-0.174369,-0.177370,-0.181068,-0.191569,-0.189557,1.115582,CHIMALTENANGO,SAN MARTIN JILOTEPEQUE,Residenciales,Presunto Fraude,-0.372017,1
2,2405770,-0.059326,-0.025614,-0.013796,0.049996,0.148672,0.141576,-0.094504,-0.100509,-0.062007,-0.027617,0.060672,0.036325,2.045466,SOLOLA,SANTA LUCIA UTATLAN,Residenciales,NO REGSITRADO,-0.372017,1
3,2442128,-0.106637,-0.101065,-0.076661,0.103363,0.042292,0.026448,0.012538,0.019419,0.047036,0.014645,0.046500,0.002842,1.430083,QUICHE,SANTA CRUZ DEL QUICHE,Residenciales,NO REGSITRADO,-0.372017,1
4,3161851,-0.136808,-0.145207,-0.161381,-0.157519,-0.152694,-0.165037,-0.156788,-0.150032,-0.140452,-0.147546,-0.151894,-0.151619,0.053211,SANTA ROSA,CUILAPA,Residenciales,Tapa rota/falta,-0.372017,1


### encode categories

In [24]:
ohe_features = scaled_features

In [25]:
ohe_NombreDepartamentoGeografia = OneHotEncoder()
ohe_NombreMunicipioGeografia = OneHotEncoder()
ohe_seccion = OneHotEncoder()
ohe_DescripcionAnomalia = OneHotEncoder()

transformed_NombreDepartamentoGeografia = ohe_NombreDepartamentoGeografia.fit_transform(ohe_features[['NombreDepartamentoGeografia']])
ohe_features[ohe_NombreDepartamentoGeografia.categories_[0]] = transformed_NombreDepartamentoGeografia.toarray()

transformed_NombreMunicipioGeografia = ohe_NombreMunicipioGeografia.fit_transform(ohe_features[['NombreMunicipioGeografia']])
ohe_features[ohe_NombreMunicipioGeografia.categories_[0]] = transformed_NombreMunicipioGeografia.toarray()

transformed_seccion = ohe_seccion.fit_transform(ohe_features[['seccion']])
ohe_features[ohe_seccion.categories_[0]] = transformed_seccion.toarray()

transformed_DescripcionAnomalia = ohe_DescripcionAnomalia.fit_transform(ohe_features[['DescripcionAnomalia']])
ohe_features[ohe_DescripcionAnomalia.categories_[0]] = transformed_DescripcionAnomalia.toarray()

ohe_features = ohe_features.drop(['NombreDepartamentoGeografia','NombreMunicipioGeografia','seccion','DescripcionAnomalia'], axis = 1)

/tmp/ipykernel_299/498564259.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ohe_features[ohe_NombreMunicipioGeografia.categories_[0]] = transformed_NombreMunicipioGeografia.toarray()
/tmp/ipykernel_299/498564259.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ohe_features[ohe_NombreMunicipioGeografia.categories_[0]] = transformed_NombreMunicipioGeografia.toarray()
/tmp/ipykernel_299/498564259.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

In [26]:
#fradues_df = pd.read_csv('data/data_final.csv',nrows=100)
#fradues_df = pd.get_dummies(fradues_df, columns = ['DEPARTAMENTO','MUNICIPIO'])

y = ohe_features['fraude']

X = ohe_features#.drop(['Unnamed: 0'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = X_train.drop(['fraude','LlavePuntoConsumo'], axis = 1)

nis_x_test = X_test['LlavePuntoConsumo']

X_test = X_test.drop(['fraude','LlavePuntoConsumo'], axis = 1)

X_test_full = X_test


In [27]:



def yield_artifacts(run_id, path=None):
    """Yield all artifacts in the specified run"""
    client = mlflow.tracking.MlflowClient()
    for item in client.list_artifacts(run_id, path):
        if item.is_dir:
            yield from yield_artifacts(run_id, item.path)
        else:
            yield item.path


def fetch_logged_data(run_id):
    """Fetch params, metrics, tags, and artifacts in the specified run"""
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    # Exclude system tags: https://www.mlflow.org/docs/latest/tracking.html#system-tags
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = list(yield_artifacts(run_id))
    return {
        "params": data.params,
        "metrics": data.metrics,
        "tags": tags,
        "artifacts": artifacts,
    }


    



In [31]:
def main():
    mlflow.set_experiment("Energuate-Regiones-especiales-supervisado")
    mlflow.sklearn.autolog()

    iris = datasets.load_iris()
    parameters = {"n_jobs":[-1],  "penalty": ('l1','l2','elasticnet','none'), "C":(1.0,10,100)}

    lr = LogisticRegression(solver = "saga")
    clf = GridSearchCV(lr, parameters, cv = 6, verbose=2)

    clf.fit(X_train.values, y_train.values)
    run_id = mlflow.last_active_run().info.run_id

    # show data logged in the parent run
    print("========== parent run ==========")
    for key, data in fetch_logged_data(run_id).items():
        print("\n---------- logged {} ----------".format(key))
        pprint(data)

    # show data logged in the child runs
    filter_child_runs = "tags.mlflow.parentRunId = '{}'".format(run_id)
    runs = mlflow.search_runs(filter_string=filter_child_runs)
    param_cols = ["params.{}".format(p) for p in parameters.keys()]
    metric_cols = ["metrics.mean_test_score"]

    print("\n========== child runs ==========\n")
    pd.set_option("display.max_columns", None)  # prevent truncating columns
    print(runs[["run_id", *param_cols, *metric_cols]])
    
    with mlflow.start_run(run_id=run_id) as run:
        print('---TEST_METRICS----')
        y_pred = clf.predict(X_test.values)
        y_true = y_test.values
        print(y_pred)
        print(y_true)
        f1 = f1_score(y_true, y_pred, average='binary')
        log_metric("test_f1_score", f1)
        print('test_score_f1 '+str(f1))
        recall = recall_score(y_true, y_pred, average='binary')
        log_metric("test_recall_score", recall)
        print('test_recall_f1 '+str(recall))
        accuracy = accuracy_score(y_true, y_pred)
        log_metric("test_accuracy_score", accuracy)
        print('test_score_accuracy '+str(accuracy))
        precision = precision_score(y_true, y_pred)
        log_metric("test_precision_score", precision)
        print('test_score_precision '+str(precision))
        #log_param("estimator", "LogisticReg")
    
        test_cfm = confusion_matrix(y_true, y_pred)
        df_cfm = pd.DataFrame(test_cfm, index = ["0", "1"], columns = ["0", "1"])
        plt.figure(figsize = (10,7))
        cfm_plot = sn.heatmap(df_cfm, annot=True)
        cfm_plot.figure.savefig("cfm_test.png")
        print(test_cfm)
        mlflow.log_artifact("cfm_test.png")


if __name__ == "__main__":
    main()

2022/10/26 08:28:37 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '40bec862e4934190b139d7062ff02083', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Fitting 6 folds for each of 12 candidates, totalling 72 fits
[CV] END .......................C=1.0, n_jobs=-1, penalty=l1; total time=   1.2s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l1; total time=   1.2s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l1; total time=   1.2s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l1; total time=   1.2s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l1; total time=   1.2s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l1; total time=   1.2s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l2; total time=   1.1s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l2; total time=   1.1s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l2; total time=   1.1s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l2; total time=   1.1s
[CV] END .......................C=1.0, n_jobs=-1, penalty=l2; total time=   1.1s
[CV] END .......................C=1.0, n_jobs=-1

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END ......................C=10, n_jobs=-1, penalty=none; total time=   1.1s


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


...

[CV] END .....................C=100, n_jobs=-1, penalty=none; total time=   1.1s


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END .....................C=100, n_jobs=-1, penalty=none; total time=   1.1s


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
18 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/mlflow/utils/autologging_utils/safety.py", line 555, in safe_patch_function
    patch_function(call_original, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/mlflow/utils/autologging_utils/safety.py", line 254, in patch_with_managed_run
    result = pa

========== parent run ==========

---------- logged params ----------
{'best_C': '10',
 'best_n_jobs': '-1',
 'best_penalty': 'none',
 'cv': '6',
 'error_score': 'nan',
 'estimator': "LogisticRegression(solver='saga')",
 'n_jobs': 'None',
 'param_grid': "{'n_jobs': [-1], 'penalty': ('l1', 'l2', 'elasticnet', "
               "'none'), 'C': (1.0, 10, 100)}",
 'pre_dispatch': '2*n_jobs',
 'refit': 'True',
 'return_train_score': 'False',
 'scoring': 'None',
 'verbose': '2'}

---------- logged metrics ----------
{'best_cv_score': 0.7502387764749859,
 'training_accuracy_score': 0.7600478468899522,
 'training_f1_score': 0.7591020127286345,
 'training_log_loss': 0.5147233151539388,
 'training_precision_score': 0.7600083110952569,
 'training_recall_score': 0.7600478468899522,
 'training_roc_auc_score': 0.828378232718911,
 'training_score': 0.7600478468899522}

---------- logged tags ----------
{'estimator_class': 'sklearn.model_selection._search.GridSearchCV',
 'estimator_name': 'GridSearchCV'

### Test model

In [ ]:
model = mlflow.sklearn.load_model('runs:/0280f97371a24d96a32e87f1e703243f/artifacts/best_estimator/')


y_pred = model.predict(X_test)
X_test_full['nis_rad'] = nis_x_test
X_test_full['fraude'] = y_pred
X_test_full.head()

### IRIS test

In [24]:
def main():
    mlflow.set_experiment("LogisticReg-test-1")
    mlflow.sklearn.autolog()

    iris = datasets.load_iris()
    parameters = {"n_jobs":[-1], "penalty": ('l2','none')}
    lr = LogisticRegression()
    clf = GridSearchCV(lr, parameters, cv = 5, verbose = 1)

    clf.fit(iris.data, iris.target)
    run_id = mlflow.last_active_run().info.run_id

    # show data logged in the parent run
    print("========== parent run ==========")
    for key, data in fetch_logged_data(run_id).items():
        print("\n---------- logged {} ----------".format(key))
        pprint(data)

    # show data logged in the child runs
    filter_child_runs = "tags.mlflow.parentRunId = '{}'".format(run_id)
    runs = mlflow.search_runs(filter_string=filter_child_runs)
    param_cols = ["params.{}".format(p) for p in parameters.keys()]
    metric_cols = ["metrics.mean_test_score"]

    print("\n========== child runs ==========\n")
    pd.set_option("display.max_columns", None)  # prevent truncating columns
    print(runs[["run_id", *param_cols, *metric_cols]])
    
    
    with mlflow.start_run(run_id=run_id) as run:
        print('---TEST_METRICS----')
        y_pred = clf.predict(iris.data)
        y_true = iris.target
        print(y_pred)
        print(y_true)
        f1 = f1_score(y_true, y_pred, average='micro')
        log_metric("test_f1_score", f1)
        print('test_score_f1 '+str(f1))
        recall = recall_score(y_true, y_pred, average='micro')
        log_metric("test_recall_score", recall)
        print('test_recall_f1 '+str(recall))
        accuracy = accuracy_score(y_true, y_pred)
        log_metric("test_accuracy_score", accuracy)
        print('test_score_accuracy '+str(accuracy))
    
        test_cfm = confusion_matrix(y_true, y_pred)
        df_cfm = pd.DataFrame(test_cfm, index = ["0", "1","2"], columns = ["0", "1","2"])
        plt.figure(figsize = (10,7))
        cfm_plot = sn.heatmap(df_cfm, annot=True)
        cfm_plot.figure.savefig("cfm_test.png")
        print(test_cfm)
        mlflow.log_artifact("cfm_test.png")

if __name__ == "__main__":
    main()
    

2022/07/15 03:45:18 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e77a44b6a41b448bb81dd678b0c9ac5a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Fitting 5 folds for each of 2 candidates, totalling 10 fits


2022/07/15 03:45:28 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.


========== parent run ==========

---------- logged params ----------
{'best_n_jobs': '-1',
 'best_penalty': 'l2',
 'cv': '5',
 'error_score': 'nan',
 'estimator': 'LogisticRegression()',
 'n_jobs': 'None',
 'param_grid': "{'n_jobs': [-1], 'penalty': ('l2', 'none')}",
 'pre_dispatch': '2*n_jobs',
 'refit': 'True',
 'return_train_score': 'False',
 'scoring': 'None',
 'verbose': '1'}

---------- logged metrics ----------
{'best_cv_score': 0.9733333333333334,
 'training_accuracy_score': 0.9733333333333334,
 'training_f1_score': 0.9733226623982927,
 'training_log_loss': 0.1194550699129196,
 'training_precision_score': 0.9738247863247864,
 'training_recall_score': 0.9733333333333334,
 'training_roc_auc_score': 0.9984,
 'training_score': 0.9733333333333334}

---------- logged tags ----------
{'estimator_class': 'sklearn.model_selection._search.GridSearchCV',
 'estimator_name': 'GridSearchCV'}

---------- logged artifacts ----------
['best_estimator/MLmodel',
 'best_estimator/conda.yaml',
 'b